In [ ]:
import numpy as np
import flammkuchen as fl
import napari
from pathlib import Path

from split_dataset import SplitDataset
import json

import flammkuchen as fl 
import tifffile as tiff

In [ ]:
master_path = Path(r"\\portulab.synology.me\data\Hagar and Ot\E0040\v10\LS")
fish_list = list(master_path.glob("*_f*"))

unexported = [f for f in fish_list if not (f / "regions_masks.h5").exists()]

path = unexported[0]
print(path)
print(len(unexported))


In [ ]:
suite2p_data = fl.load(path / "data_from_suite2p_cells.h5")
traces = suite2p_data['traces']
roi_stack = suite2p_data['rois_stack']
all_coords = suite2p_data['coords']

In [ ]:
anatomy = suite2p_data['anatomy_stack']

In [ ]:
viewer = napari.view_image(anatomy,  colormap="gray")
#viewer.add_image(roi_stack, colormap="red", blending="additive")
ipn = np.zeros(np.shape(anatomy), dtype=int)
habenula_r = np.zeros(np.shape(anatomy), dtype=int)
habenula_l = np.zeros(np.shape(anatomy), dtype=int)
ahb = np.zeros(np.shape(anatomy), dtype=int)
pretectum = np.zeros(np.shape(anatomy), dtype=int)


rhab_layer = viewer.add_labels(habenula_r, name='habenula_r')
lhab_layer = viewer.add_labels(habenula_l, name='habenula_l')
ipn_layer = viewer.add_labels(ipn, name='ipn')
ahb_layer = viewer.add_labels(ahb, name='ahb')
pretectum_layer = viewer.add_labels(pretectum, name='pretectum')

In [ ]:
#### Run the next cells only after marking the regions you want to keep in napari

In [ ]:
rhab_layer = viewer.layers["habenula_r"].data
lhab_layer = viewer.layers["habenula_l"].data
ipn_layer = viewer.layers["ipn"].data
ahb_layer = viewer.layers["ahb"].data
pretectum_layer = viewer.layers["pretectum"].data

In [ ]:
masks = np.copy(rhab_layer)
masks = masks + (lhab_layer * 2) + (ipn_layer * 3) + (ahb_layer * 4) + (pretectum_layer * 5)

In [ ]:
regions = [rhab_layer, lhab_layer, ipn_layer, ahb_layer, pretectum_layer]
coords_to_keep = [[], [], [], [], []]
num_cells = np.shape(all_coords)[0]


for region in range(len(regions)): 
    tmp_coords_to_keep = []
    labels = regions[region]
    for i in range(num_cells):
        tmp_cell = labels[int(all_coords[i, 0]), int(all_coords[i, 1]), int(all_coords[i, 2])]
        if tmp_cell > 0:
            tmp_coords_to_keep = tmp_coords_to_keep + [i]
            
    coords_to_keep[region] = tmp_coords_to_keep

In [ ]:
print(len(coords_to_keep[0]))

In [ ]:
d = {
    'l_habenula_coords': coords_to_keep[0],
    'r_habenula_coords': coords_to_keep[1],
    'ipn_coords': coords_to_keep[2],
    'ahb_coords': coords_to_keep[3],
    'pretectum_coords': coords_to_keep[4],
    'masks': masks,
}
fl.save(path / 'regions_masks.h5', d)


In [ ]:
np.unique(masks)